In [1]:
from commons import *
import openml

In [2]:
datasetIds = sorted(pd.unique(pd.read_csv("data/datasets.csv")["openmlid"]))

In [3]:
datasets = openml.datasets.get_datasets([int(x) for x in datasetIds],download_data=False)

In [4]:
print(datasets[0].default_target_attribute)

class


# Create Dataset Overview (LaTex)
The considered features are:
 * number of instances(ni)
 * number of attributes in total (na)
 * numeric  (nn)
 * symbolic/categorical  (ns),
 * after  binarization(nab)
 * number  of  labels  (nl), 
 * number  of  possible  values for  the  nominal  attributes  (nc),
 * total  variance  (tv), 
 * attributes  required  to  cover  50%,  90%,  95%,  and  90%  ofthe variance (vx).

In [5]:
mfFile = "data/metafeatures-derived.csv"

In [21]:
dfMyTest = pd.read_csv("data/runtimes/runtimes_atomic_nooutliers.csv")

/home/felix/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
dfMyTest.query("openmlid == 41991")

,openmlid,totalsize,algorithm,algorithmoptions,seed,fitsize,numattributes,numlabels,numnumericattributes,numsymbolicattributes,numberofcategories,numericattributesafterbinarization,totalvariance,attributestocover50pctvariance,attributestocover90pctvariance,attributestocover95pctvariance,attributestocover99pctvariance,applicationsize,fittime,applicationtime
105053,41991,270912,decisiontable,NaN,0,2,784,2.0,784.0,0.0,0.0,784.0,5.387466e+06,85.0,180.0,204.0,243.0,270910,1,3434.0
105054,41991,270912,kstar,NaN,0,2,784,2.0,784.0,0.0,0.0,784.0,5.387466e+06,85.0,180.0,204.0,243.0,270910,1,459.0
105055,41991,270912,multilayerperceptron,NaN,0,2,784,2.0,784.0,0.0,0.0,784.0,5.387466e+06,85.0,180.0,204.0,243.0,270910,8,1220.0
105056,41991,270912,naivebayes,NaN,0,2,784,2.0,784.0,0.0,0.0,784.0,5.387466e+06,85.0,180.0,204.0,243.0,270910,1,3909.0
105057,41991,270912,oner,NaN,0,2,784,2.0,784.0,0.0,0.0,784.0,5.387466e+06,85.0,180.0,204.0,243.0,270910,1,3792.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028162,41991,270912,oner,-B 16,1,100000,784,49.0,784.0,0.0,0.0,784.0,5.338014e+06,255.0,543.0,606.0,699.0,170912,551,1098.0
1028163,41991,270912,oner,-B 32,1,100000,784,49.0,784.0,0.0,0.0,784.0,5.338014e+06,255.0,543.0,606.0,699.0,170912,155,792.0
1028164,41991,270912,multilayerperceptron,-B,2,100000,784,49.0,784.0,0.0,0.0,784.0,5.343749e+06,255.0,544.0,607.0,699.0,170912,3600,NaN
1028165,41991,270912,oner,-B 32,2,100000,784,49.0,784.0,0.0,0.0,784.0,5.343749e+06,255.0,544.0,607.0,699.0,170912,169,816.0


In [49]:
dfMetafeatures = pd.read_csv(mfFile) if path.exists(mfFile) else  pd.DataFrame([], columns=["openmlid", "name", "numinstances", "numattributes", "numericattributes", "categoricalattributes", "attributesafterbinarization", "numlabels", "numcategories", "totalvariance", "v50", "v90", "v95", "v99"])

In [46]:
# Print a summary
for dataset in tqdm(datasets):
    
    if len(dfMetafeatures[dfMetafeatures["openmlid"] == dataset.id]) > 0:
        continue
    if dataset.id == 41991:
        continue
    print(dataset.id)
    print(dataset.name)
    
    # now compute the features used in the paper:
    dfData = dataset.get_data()[0]
    target = dataset.default_target_attribute
    shape = dfData.shape
    dfData.fillna(value=0, inplace=True)
    
    # get feature types
    print("\tComputing number of types (there are " + str(len(dfData.columns)) + " attributes)")
    typeTable = dfData.dtypes
    atts_numeric = 0
    atts_categorical = 0
    num_categories = 0
    num_binaryattributes = 0
    colsToBinarize = []
    for i, t in enumerate(typeTable):
        if i == list(dfData.columns).index(target):
            continue
        if type(t) == pd.core.dtypes.dtypes.CategoricalDtype:
            atts_categorical += 1
            colsToBinarize.append(dfData.columns[i])
            if len(t.categories) > 2: # only augment values for non-binary attributes
                num_categories += len(t.categories)
            else:
                num_binaryattributes += 1
        else:
            if type(t) == np.dtype:
                atts_numeric += 1
                
    if len(colsToBinarize) > 0:
        print("\tComputing number of attributes after binarization. There are " + str(len(colsToBinarize)) + " cols to be binarized!")
        dfDataBinarized = pd.get_dummies(dfData.drop(columns=target), columns=colsToBinarize)
        dfDataBinarized.fillna(value=pd.np.nan, inplace=True)
        num_attributes_afterbinarization = dfDataBinarized.shape[1]# atts_numeric + num_binaryattributes + num_categories
    else:
        print("Not binarizing since all attributes are numeric.")
        num_attributes_afterbinarization = dfData.shape[1] - 1
        dfDataBinarized = dfData.drop(columns=target)
    num_labels = len(pd.unique(dfData[target]))
    
    print("\tComputing variances of columns")
    #for v in dfDataBinarized.values[:,0]:
    variances = sorted(np.nanvar(dfDataBinarized.values, axis=0))
    print("Done, now computing the variance metrics ...")
    #variances = variances[np])
    variances = [v for v in variances if not np.isnan(v)]
    print(variances)
    variances.reverse()
    total_variance = np.sum(variances)
    if np.isnan(total_variance):
        raise Exception()
    cov = 0
    var50 = None
    var90 = None
    var95 = None
    var99 = None
    for i, var in enumerate(variances):
        cov += var / total_variance
        if var50 is None and cov >= .5:
            var50 = i + 1
        if var90 is None and cov >= .9:
            var90 = i + 1
        if var95 is None and cov >= .95:
            var95 = i + 1
        if var99 is None and cov >= .99:
            var99 = i + 1
    print(var50)
    
    row = [dataset.id, dataset.name, shape[0], shape[1] - 1, atts_numeric, atts_categorical, num_attributes_afterbinarization, num_labels, num_categories, total_variance, var50, var90, var95, var99]
    dfMetafeatures.loc[len(dfMetafeatures)] = row
    dfMetafeatures.to_csv(mfFile, index=False)

273
IMDB.drama
	Computing number of types (there are 1002 attributes)
Not binarizing since all attributes are numeric.
	Computing variances of columns
Done, now computing the variance metrics ...
[0.0044950303, 0.0045032254, 0.0047326367, 0.0049128137, 0.0049373778, 0.0050110645, 0.005011065, 0.0051420326, 0.0052402387, 0.0052893315, 0.005379327, 0.0056819133, 0.0057146135, 0.005780008, 0.005902602, 0.0059924833, 0.0060660094, 0.006106852, 0.0061150207, 0.0062620263, 0.006335513, 0.006343679, 0.006351843, 0.0064008245, 0.0064008245, 0.006433477, 0.006645667, 0.0066619865, 0.0066701453, 0.006686461, 0.006776199, 0.006816983, 0.0068659196, 0.006939314, 0.007037159, 0.0070860735, 0.007102378, 0.007143131, 0.0071920375, 0.007240936, 0.007265383, 0.007273532, 0.0072816806, 0.0073061245, 0.007306125, 0.007355011, 0.007371306, 0.0073794527, 0.0073794527, 0.0073957467, 0.007444626, 0.007460917, 0.0074853515, 0.00750979, 0.007542367, 0.00755051, 0.007558655, 0.0075586555, 0.0075993743, 0.007607

In [62]:
dfLatex = dfMetafeatures.dropna().copy().astype({"v50": int, "v90": int, "v95": int, "v99": int}).sort_values("openmlid")
dfLatex["totalvariance"] = np.round(dfLatex["totalvariance"], 2)
dfLatex["name"] = [n[:12] + "..." if len(n) > 15 else n for n in dfLatex["name"]]

# compute number of considered fit sizes
fitsizes = [2, 4, 8, 16, 50, 75, 100, 125, 150, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 60000, 70000, 80000, 90000, 100000]
fsCol = []
for i, row in dfLatex.iterrows():
    relevant = [fs for fs in fitsizes if fs < row["numinstances"]]
    fsCol.append(len(relevant))
dfLatex["fs"] = fsCol

In [63]:

print(dfLatex.to_latex(index=False, float_format="%.2E"))

\begin{tabular}{rlrrrrrrrrrrrrr}
\toprule
 openmlid &             name &  numinstances &  numattributes &  numericattributes &  categoricalattributes &  attributesafterbinarization &  numlabels &  numcategories &  totalvariance &    v50 &    v90 &    v95 &    v99 &  fs \\
\midrule
        3 &         kr-vs-kp &          3196 &             37 &                  0 &                     36 &                           75 &          2 &              6 &       9.49E+00 &     21 &     46 &     53 &     62 &  23 \\
        6 &           letter &         20000 &             17 &                 16 &                      0 &                           17 &         26 &              0 &       8.55E+01 &      6 &     14 &     15 &     16 &  33 \\
       12 &    mfeat-factors &          2000 &            217 &                216 &                      0 &                          217 &         10 &              0 &       9.89E+05 &     34 &     85 &     95 &    107 &  20 \\
       14 &    mfeat-four

In [16]:
tex = "\\begin{longtable}{rlrrrrrrrrrrrr}\n"
tex += "\tID & Name & ni & na & nn & ns & nab & nl & nc & tv & v50 & v90 & v95 & v99\\\\\\hline\n"

    tex += "\t"
    tex += "\\href{http://openml.org/d/" + str(dataset.id) + "}{" + str(dataset.id) + "}"
    tex += "&"
    tex += dataset.name.replace("_", "\\_")
    tex += "&"
    
    print(dataset.name)

IndentationError: unexpected indent (<ipython-input-16-a267ec4220f9>, line 4)

In [38]:
pyperclip.copy("test")